In [1]:
import sys

sys.path.append("../..")
import pandas as pd
from src.data_loaders.rollcall import (
    get_raw_individual_votes, 
    get_raw_party_membership, 
    get_individual_votes_with_party_enriched, 
    get_individual_votes_with_party_enriched,
    get_trainig_data_v4,
    get_trainig_data_v3
)

In [15]:
df = get_raw_party_membership()
df_dedupe = df[["icpsr", "congress"]].drop_duplicates().sort_values(["icpsr", "congress"])
df["terms_served"] = (df_dedupe.groupby("icpsr")["congress"].cumcount() + 1).to_frame("terms_served")
df["terms_served"].value_counts().head()

terms_served
1.0    12667
2.0     8967
3.0     6452
4.0     4789
5.0     3799
Name: count, dtype: int64

In [16]:
df.columns

Index(['congress', 'chamber', 'icpsr', 'state_icpsr', 'district_code',
       'state_abbrev', 'party_code', 'occupancy', 'last_means', 'bioname',
       'bioguide_id', 'born', 'died', 'nominate_dim1', 'nominate_dim2',
       'nominate_log_likelihood', 'nominate_geo_mean_probability',
       'nominate_number_of_votes', 'nominate_number_of_errors', 'conditional',
       'nokken_poole_dim1', 'nokken_poole_dim2', 'terms_served'],
      dtype='object')

In [3]:
# Confirmed that John Dingell was the longest serving congresssman in the house
df.loc[df["terms_served"] == df["terms_served"].max()].T.loc["bioname"]

47387    DINGELL, John David, Jr.
Name: bioname, dtype: object

In [4]:
house_df = df.loc[df["chamber"] == "House"]
district_count = house_df.groupby(["state_abbrev", "state_icpsr", "congress"])["district_code"].nunique().to_frame('num_reps')
total_district_count = district_count.groupby("congress")["num_reps"].sum().to_frame('num_reps_total')
count_df = district_count.join(total_district_count)

count_df["pct_pop"] = count_df["num_reps"] / count_df["num_reps_total"]
count_df.xs(118, level="congress").sort_values("pct_pop", ascending=False).head()

,,num_reps,num_reps_total,pct_pop
state_abbrev,state_icpsr,,,
CA,71,52,441,0.117914
TX,49,38,441,0.086168
FL,43,28,441,0.063492
NY,13,26,441,0.058957
IL,21,17,441,0.038549


In [5]:
df = df.set_index(["state_icpsr", "congress"]).join(count_df.droplevel('state_abbrev')).reset_index()


In [10]:
df = get_individual_votes_with_party_enriched()
df.head()

Loading cached data from /Users/declannelson/Desktop/columbia/stat5241/stat5241_team_7/src/data/individual_votes_8.parquet


date bill_number  \
congress session chamber rollnumber icpsr                           
101      2.0     0       371        633.0  1990-01-24      hr2712   
                                    1077.0 1990-01-24      hr2712   
                                    1087.0 1990-01-24      hr2712   
                                    2009.0 1990-01-24      hr2712   
                                    2605.0 1990-01-24      hr2712   

                                            vote_passed  vote_type_amend  \
congress session chamber rollnumber icpsr                                  
101      2.0     0       371        633.0             1                0   
                                    1077.0            1                0   
                                    1087.0            1                0   
                                    2009.0            1                0   
                                    2605.0            1                0   

                                            vote_type_cloture  \
congress session chamber rollnumber icpsr                       
101      2.0     0       371        633.0                   0   
                                    1077.0                  0   
                                    1087.0                  0   
                                    2009.0                  0   
                                    2605.0                  0   

                                            vote_type_concur  \
congress session chamber rollnumber icpsr                      
101      2.0     0       371        633.0                  0   
                                    1077.0                 0   
                                    1087.0                 0   
                                    2009.0                 0   
                                    2605.0                 0   

                                            vote_type_conference  \
congress session chamber rollnumber icpsr                          
101      2.0     0       371        633.0                      0   
                                    1077.0                     0   
                                    1087.0                     0   
                                    2009.0                     0   
                                    2605.0                     0   

                                            vote_type_pass  \
congress session chamber rollnumber icpsr                    
101      2.0     0       371        633.0                0   
                                    1077.0               0   
                                    1087.0               0   
                                    2009.0               0   
                                    2605.0               0   

                                            vote_type_recommit  \
congress session chamber rollnumber icpsr                        
101      2.0     0       371        633.0                    0   
                                    1077.0                   0   
                                    1087.0                   0   
                                    2009.0                   0   
                                    2605.0                   0   

                                            vote_type_suspend  ...  \
congress session chamber rollnumber icpsr                      ...   
101      2.0     0       371        633.0                   0  ...   
                                    1077.0                  0  ...   
                                    1087.0                  0  ...   
                                    2009.0                  0  ...   
                                    2605.0                  0  ...   

                                            crs_policy_area_taxation  \
congress session chamber rollnumber icpsr                              
101      2.0     0       371        633.0                          0   
                                    1077.0   

In [13]:
prior_votes = df.sort_index().groupby(["congress", "rollnumber", "icpsr"])["vote_for"].cumcount().to_frame("prior_votes_for_bill")
prior_votes.head()

prior_votes_for_bill
congress session chamber rollnumber icpsr                       
101      1.0     1       11         660.0                      0
                                    1252.0                     0
                                    1366.0                     0
                                    4812.0                     0
                                    9369.0                     0

In [14]:
df.join(prior_votes)

date bill_number  \
congress session chamber rollnumber icpsr                            
101      2.0     0       371        633.0   1990-01-24      hr2712   
                                    1077.0  1990-01-24      hr2712   
                                    1087.0  1990-01-24      hr2712   
                                    2009.0  1990-01-24      hr2712   
                                    2605.0  1990-01-24      hr2712   
...                                                ...         ...   
115      2.0     1       350        49308.0 2018-01-29       s2311   
                                    49703.0 2018-01-29       s2311   
                                    49706.0 2018-01-29       s2311   
                                    94659.0 2018-01-29       s2311   
                                    99912.0 2018-01-29       s2311   

                                             vote_passed  vote_type_amend  \
congress session chamber rollnumber icpsr                                   
101      2.0     0       371        633.0              1                0   
                                    1077.0             1                0   
                                    1087.0             1                0   
                                    2009.0             1                0   
                                    2605.0             1                0   
...                                                  ...              ...   
115      2.0     1       350        49308.0            0                0   
                                    49703.0            0                0   
                                    49706.0            0                0   
                                    94659.0            0                0   
                                    99912.0            0                0   

                                             vote_type_cloture  \
congress session chamber rollnumber icpsr                        
101      2.0     0       371        633.0                    0   
                                    1077.0                   0   
                                    1087.0                   0   
                                    2009.0                   0   
                                    2605.0                   0   
...                                                        ...   
115      2.0     1       350        49308.0                  1   
                                    49703.0                  1   
                                    49706.0                  1   
                                    94659.0                  1   
                                    99912.0                  1   

                                             vote_type_concur  \
congress session chamber rollnumber icpsr                       
101      2.0     0       371        633.0                   0   
                                    1077.0                  0   
                                    1087.0                  0   
                                    2009.0                  0   
                                    2605.0                  0   
...                                                       ...   
115      2.0     1       350        49308.0                 0   
                                    49703.0                 0   
                                    49706.0                 0   
                                    94659.0                 0   
                                    99912.0                 0   

                                             vote_type_conference  \
congress session chamber rollnumber icpsr                           
101      2.0     0       371        633.0                       0   
                                    1077.0                      0   
                                    1087.0                      0   
                                    2009.0                      0   
                  

In [2]:
features, target_col, df = get_trainig_data_v3()

Loading cached data from /Users/declannelson/Desktop/columbia/stat5241/stat5241_team_7/src/data/individual_votes_9.parquet


In [3]:
df.isna().sum()

vote_for                                                      0
icpsr                                                         0
congress                                                      0
chamber                                                       0
session                                                       0
d                                                             0
r                                                             0
terms_served                                                  0
pct_pop                                                       0
prior_votes_for_bill                                          0
vote_type_amend                                               0
vote_type_cloture                                             0
vote_type_concur                                              0
vote_type_conference                                          0
vote_type_pass                                                0
vote_type_recommit                      

In [4]:
df = pd.read_csv("../data/dime_recipients_1979_2024.csv")

/var/folders/y7/rdkd1_7d3b9fjykc4g2vvjl00000gn/T/ipykernel_76763/1558342054.py:1: DtypeWarning: Columns (10,12,15,16,17,41,43,44,45,47,49,50,54,55,58,59,60,61,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/dime_recipients_1979_2024.csv")


In [5]:
df.shape

(479502, 64)

In [6]:
df.columns

Index(['election', 'cycle', 'fecyear', 'bonica.rid', 'bonica.cid', 'name',
       'lname', 'ffname', 'fname', 'mname', 'title', 'suffix', 'party',
       'state', 'seat', 'district', 'distcyc', 'ico.status', 'cand.gender',
       'recipient.cfscore', 'recipient.cfscore.dyn', 'contributor.cfscore',
       'dwdime', 'dwnom1', 'dwnom2', 'ps.dwnom1', 'ps.dwnom2', 'irt.cfscore',
       'composite.score', 'num.givers', 'num.givers.total', 'total.receipts',
       'total.disbursements', 'total.indiv.contribs', 'total.unitemized',
       'total.pac.contribs', 'total.party.contribs',
       'total.contribs.from.candidate', 'ind.exp.support', 'ind.exp.oppose',
       'prim.vote.pct', 'pwinner', 'gen.vote.pct', 'gwinner', 's.elec.stat',
       'r.elec.stat', 'district.pres.vs', 'fec.cand.status', 'recipient.type',
       'igcat', 'comtype', 'ICPSR', 'ICPSR2', 'Cand.ID', 'FEC.ID', 'NID',
       'before.switch.ICPSR', 'after.switch.ICPSR', 'party.orig',
       'nimsp.party', 'nimsp.candidate.ICO.co

In [21]:
df.sort_values("cycle").tail(1000).to_csv("../data/dime_example.csv")

In [18]:
df

,election,cycle,fecyear,bonica.rid,bonica.cid,name,lname,ffname,fname,mname,...,FEC.ID,NID,before.switch.ICPSR,after.switch.ICPSR,party.orig,nimsp.party,nimsp.candidate.ICO.code,nimsp.district,nimsp.office,nimsp.candidate.status
0,fd1980,1980,1980.0,cand2,2.262643e+09,"whitten, jamie lloyd",whitten,jamie lloyd,jamie,lloyd,...,C00094912,N00003287,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
1,fd1982,1982,1982.0,cand2,2.262643e+09,"whitten, jamie lloyd",whitten,jamie lloyd,jamie,lloyd,...,C00094912,N00003287,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
2,fd1984,1984,1984.0,cand2,2.262643e+09,"whitten, jamie lloyd",whitten,jamie lloyd,jamie,lloyd,...,C00094912,N00003287,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
3,fd1986,1986,1986.0,cand2,2.262643e+09,"whitten, jamie lloyd",whitten,jamie lloyd,jamie,lloyd,...,C00094912,N00003287,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
4,fd1988,1988,1988.0,cand2,2.262643e+09,"whitten, jamie lloyd",whitten,jamie lloyd,jamie,lloyd,...,C00094912,N00003287,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479497,OH1998,1998,1998.0,cand95076,1.000342e+08,"petro, jim",petro,jim,jim,NaN,...,NaN,NaN,NaN,NaN,200,REPUBLICAN,I,SW,auditor,Won
479498,OH2006,2006,2006.0,cand95348,5.000004e+15,"sykes, barbara",sykes,barbara,barbara,NaN,...,NaN,NaN,NaN,NaN,100,DEMOCRAT,IO,SW,auditor,Lost - General Election
479499,OH2006,2006,2006.0,cand95395,5.000004e+15,"taylor, mary",taylor,mary,mary,NaN,...,NaN,NaN,NaN,NaN,200,REPUBLICAN,IO,SW,auditor,Won
479500,OH2008,2008,2008.0,cand95395,5.000004e+15,"taylor, mary",taylor,mary,mary,NaN,...,NaN,NaN,NaN,NaN,200,REPUBLICAN,I,SW,auditor,Not Up For Election


In [22]:
df["recipient.type"].unique()

array(['cand', 'comm'], dtype=object)